In [8]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
import google.generativeai as genai
import gradio as gd
from groq import Groq

In [9]:
base_url='http://localhost:11434/v1'
ollama_client = OpenAI(base_url=base_url, api_key='ollama')
base_url_google="https://generativelanguage.googleapis.com/v1beta/openai/"
load_dotenv(override=True)
gemini_api_key = os.getenv("GOOGLE_API_KEY")
gemini_client = OpenAI(base_url=base_url_google,api_key=gemini_api_key)
base_url_groq ="https://api.groq.com/openai/v1"
groq_api_key =os.getenv("GROQ_API_KEY")
groq_client = OpenAI(base_url=base_url_groq, api_key=groq_api_key)

In [3]:
#Using streaming functionality of LLMs and Gradio

def Gemini_call(prompt):
    messages = [{
        'role':'system','content':system_prompt},
        {'role':'user', 'content':prompt}
        ]
    stream = gemini_client.chat.completions.create(
        model='gemini-2.5-flash-lite',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [4]:
def Ollama_call(prompt):
    messages = [{
        'role':'system','content':system_prompt},
        {'role':'user', 'content':prompt}
        ]
    stream = ollama_client.chat.completions.create(
        model='gemma3:latest',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [23]:
#defining groq
def Groq_call(prompt):
    messages =[{'role':'system','content':system_prompt},
               {'role':'user','content':prompt}
               ]
    stream = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages = messages, stream = True
    )
    result = ""
    for chunk in stream:
        result+= chunk.choices[0].delta.content or ""
        yield result

In [12]:
#Creating a drop down for the multiple LLMs at one place

def stream_model(prompt,model):
    if model== 'Ollama':
        result = Ollama_call(prompt)
    elif model=='Gemini':
        result = Gemini_call(prompt)
    elif model=='Groq':
        result = Groq_call(prompt)
    else:
        raise ValueError("unknown Model")
    yield from result




In [26]:
system_prompt = "You are an intellgent agent who responses in Markdown and not in code blocks"
message_input = gd.Textbox(label ='Your message',info ="Enter a message",lines=7)
model_selector =gd.Dropdown(["Gemini","Ollama","Groq"],label='Select model',value='Ollama')
message_output= gd.Markdown(label='Response:')

view = gd.Interface(
    fn= stream_model,
    title='Multiple Model at one place',
    inputs=[message_input,model_selector],
    outputs=[message_output],
    examples=[
        ['explain baking of the chcocolate cake in Indian house','Gemini'],
        ['Help me mathematically derive the linear equation','Ollama'],
        ['Convert $1000 into INR, Euros, Pounds in matrix','Groq']
    ],
    flagging_mode='never'
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.
